In [1]:
from IPython.display import display, Markdown
import torch
import torch.nn as nn
import torch
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import zipfile as zf
import glob
from PIL import Image
from sklearn.model_selection import train_test_split

In [2]:
# Seta a semente para o random

torch.manual_seed(37)

In [3]:
# Verifica dos diretórios da pasta 'data'

os.listdir('data')

['test', 'test.zip', 'train', 'train.zip']

In [4]:
# Extrai os arquivos zipados se eles já não foram extraídos

if (os.path.isdir('data/train')) == False:
    with zf.ZipFile(os.path.join('data', 'train.zip')) as train_zip:
        train_zip.extractall('data')
    
if (os.path.isdir('data/test')) == False:
    with zf.ZipFile(os.path.join('data', 'test.zip')) as test_zip:
        test_zip.extractall('data')    

In [5]:
# Cria uma lista de treinamento e de teste com todos os arquivos de extensão jpg existentes no diretório setado

train_dir = 'data/train'
test_dir = 'data/test'

train_list = glob.glob(os.path.join(train_dir, '*.jpg'))
test_list = glob.glob(os.path.join(test_dir, '*.jpg'))


In [6]:
#train_list[0].split('/')[-1].split('\\')[-1].split('.')[0]

In [7]:
# Verifica os tamanhos das listas com os arquivos de treinamento e de teste

display(Markdown("**Tamanho dos Conjuntos de Treinamento e Teste:**"))
print(len(train_list), len(test_list))

**Tamanho dos Conjuntos de Treinamento e Teste:**

25000 12500


In [8]:
# Divide a lista de treinamento em treinamento e validação e verifica o tamanho das novas listas

train_list, val_list = train_test_split(train_list, test_size=0.2)

display(Markdown("**Tamanho dos Conjuntos de Treinamento e Validação:**"))
print(len(train_list),len(val_list))

**Tamanho dos Conjuntos de Treinamento e Validação:**

20000 5000


In [9]:
# Transforma as imagens redefinindo o tamanho

train_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

val_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

test_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

In [10]:
# Seta alguns hiperparâmetros do modelo

lr = 0.001
batch_size = 300
epochs = 6

In [11]:
# Define o dataset

class dataset(Dataset):
    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform
        
    def __len__(self):
        self.fileLength = len(self.file_list)
        
        return self.fileLength
    
    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img)
        
        label = img_path.split('/')[-1].split('\\')[-1].split('.')[0]
        if label == 'dog':
            label = 1
        elif label == 'cat':
            label = 0
            
        return img_transformed, label
    

In [12]:
# Cria o dataset e o dataloader para dividir os dados de treinamento em lotes (batchs)

train_data = dataset(train_list, transform=train_transforms)
val_data = dataset(val_list, transform=val_transforms)
test_data = dataset(test_list, transform=test_transforms)

train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)

print(len(train_data),len(val_data),len(test_data))
print(len(train_loader),len(val_loader),len(test_loader))

20000 5000 12500
67 17 42


In [13]:
# Verifica o shape da imagem

train_data[0][0].shape

torch.Size([3, 224, 224])

In [14]:
# Define a Rede Neural Convolucional

class Cnn(nn.Module):
    def __init__(self):
        super(Cnn,self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=0, stride=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=0, stride=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
            
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=0, stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        
        self.fc1 = nn.Linear(3*3*64, 10)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(10, 2)
        self.relu = nn.ReLU()
        
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.relu(self.fc1(out))
        out = self.fc2(out)

        return out

In [15]:
# Instancia o modelo e o coloca em modo de treinamento

display(Markdown("**Rede Neural Convolucional:**"))
model = Cnn()
model.train()

**Rede Neural Convolucional:**

Cnn(
  (layer1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=576, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=10, out_features=2, bias=True)
  (re

In [16]:
# Instancia um otimizador e um critério de avaliação

optimizer = optim.Adam(model.parameters(), lr)
criterion = nn.CrossEntropyLoss()

In [17]:
# Treina o modelo e exibe a acurácia e o custo de treinamento e de validação para cada época

display(Markdown("**Métricas de treinamento para os conjuntos de treinamento e validação:**"))
for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0
    
    for data, label in train_loader:
        output = model(data)
        loss = criterion(output, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        acc = ((output.argmax(dim=1) == label).float().mean())
        epoch_accuracy += acc/len(train_loader)
        epoch_loss += loss/len(train_loader)
        
    print("Epoch: {}, train accuracy: {}, train loss: {}".format(epoch+1, epoch_accuracy, epoch_loss))
    
    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        
        for data, label in val_loader:
            val_output = model(data)
            val_loss = criterion(val_output, label)
            
            acc = ((val_output.argmax(dim=1) == label).float().mean())
            epoch_val_accuracy += acc/len(val_loader)
            epoch_val_loss += loss/len(val_loader)
            
        print("Epoch: {}, val accuracy: {}, val loss: {}".format(epoch+1, epoch_val_accuracy, epoch_val_loss))
            

**Métricas de treinamento para os conjuntos de treinamento e validação:**

Epoch: 1, train accuracy: 0.6295523047447205, train loss: 0.6358184218406677
Epoch: 1, val accuracy: 0.6850980520248413, val loss: 0.6035308837890625
Epoch: 2, train accuracy: 0.7058955430984497, train loss: 0.5669397711753845
Epoch: 2, val accuracy: 0.7206862568855286, val loss: 0.5407438278198242
Epoch: 3, train accuracy: 0.7238556742668152, train loss: 0.5374974608421326
Epoch: 3, val accuracy: 0.731372594833374, val loss: 0.5432770252227783
Epoch: 4, train accuracy: 0.7342786192893982, train loss: 0.5231187343597412
Epoch: 4, val accuracy: 0.7536275386810303, val loss: 0.5532534718513489
Epoch: 5, train accuracy: 0.7514676451683044, train loss: 0.5069639682769775
Epoch: 5, val accuracy: 0.7605882883071899, val loss: 0.5262418389320374
Epoch: 6, train accuracy: 0.7537065744400024, train loss: 0.4942244291305542
Epoch: 6, val accuracy: 0.759117603302002, val loss: 0.48699045181274414


In [18]:
# Seta o modelo para o modo de avaliação e solicita as predições para os dados de teste

model.eval()

probs = []
with torch.no_grad():
    for data, iFile in test_loader:
        pred = model(data)
        
        pred_list = F.softmax(pred, dim=1)[:,1].tolist()
        probs += list(zip(list(iFile), pred_list))
        

In [19]:
# Exibe as predições para cada imagem de teste

probs.sort(key = lambda x: int(x[0]))
probs

[('1', 0.8266782760620117),
 ('2', 0.935295045375824),
 ('3', 0.23815476894378662),
 ('4', 0.99595707654953),
 ('5', 0.1538296341896057),
 ('6', 0.6305022835731506),
 ('7', 0.5879395008087158),
 ('8', 0.2851681411266327),
 ('9', 0.20222589373588562),
 ('10', 0.255009263753891),
 ('11', 0.14577285945415497),
 ('12', 0.8554399013519287),
 ('13', 0.4843160808086395),
 ('14', 0.8072583079338074),
 ('15', 0.7512488961219788),
 ('16', 0.31475600600242615),
 ('17', 0.8997582197189331),
 ('18', 0.8523918390274048),
 ('19', 0.4154094159603119),
 ('20', 0.537013828754425),
 ('21', 0.9487501382827759),
 ('22', 0.13087871670722961),
 ('23', 0.28083446621894836),
 ('24', 0.6485759019851685),
 ('25', 0.5796958208084106),
 ('26', 0.8296113610267639),
 ('27', 0.7830832004547119),
 ('28', 0.11843334138393402),
 ('29', 0.693710207939148),
 ('30', 0.8460091352462769),
 ('31', 0.5920313596725464),
 ('32', 0.25846874713897705),
 ('33', 0.927428662776947),
 ('34', 0.18006713688373566),
 ('35', 0.06596113741

In [24]:
# Cria as listas de índices, probabilidades e labels preditas

idx = list(map(lambda x: x[0], probs))
prob = list(map(lambda x: x[1], probs))
pred = []
for i in range(len(prob)):
    if prob[i]<0.5:
        pred.append("cat")
        prob[i] = 1-prob[i]
    else:
        pred.append("dog")

In [26]:
# Cria um dataframe para os resultados dos dados de teste

display(Markdown("**Predições para o conjunto de teste:**"))
submission = pd.DataFrame({'id': idx, 'pred': pred, 'prob': prob})
submission

**Predições para o conjunto de teste:**

,id,pred,prob
0,1,dog,0.826678
1,2,dog,0.935295
2,3,cat,0.761845
3,4,dog,0.995957
4,5,cat,0.846170
...,...,...,...
12495,12496,cat,0.510480
12496,12497,cat,0.816078
12497,12498,cat,0.761644
12498,12499,dog,0.940175


In [28]:
train_list

['data/train\\cat.10448.jpg',
 'data/train\\dog.716.jpg',
 'data/train\\dog.6502.jpg',
 'data/train\\dog.9741.jpg',
 'data/train\\dog.727.jpg',
 'data/train\\cat.3504.jpg',
 'data/train\\dog.8011.jpg',
 'data/train\\cat.834.jpg',
 'data/train\\dog.4884.jpg',
 'data/train\\cat.4607.jpg',
 'data/train\\dog.3523.jpg',
 'data/train\\dog.32.jpg',
 'data/train\\dog.5055.jpg',
 'data/train\\dog.5383.jpg',
 'data/train\\cat.11800.jpg',
 'data/train\\dog.5725.jpg',
 'data/train\\cat.1060.jpg',
 'data/train\\dog.835.jpg',
 'data/train\\cat.7933.jpg',
 'data/train\\cat.9625.jpg',
 'data/train\\cat.8287.jpg',
 'data/train\\cat.3361.jpg',
 'data/train\\cat.12405.jpg',
 'data/train\\cat.12088.jpg',
 'data/train\\cat.3729.jpg',
 'data/train\\dog.8494.jpg',
 'data/train\\cat.390.jpg',
 'data/train\\dog.8687.jpg',
 'data/train\\dog.9755.jpg',
 'data/train\\cat.11864.jpg',
 'data/train\\dog.7991.jpg',
 'data/train\\cat.9059.jpg',
 'data/train\\cat.339.jpg',
 'data/train\\cat.2120.jpg',
 'data/train\\dog